## 1- test distilbert préentrainé

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline
import tensorflow as tf

classifier = pipeline("sentiment-analysis") #“distilbert-base-uncased-finetuned-sst-2-english
results=classifier("we love you")
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
results=classifier("we love you")
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
results=classifier("we love you")
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
    
    

## 2-test distilbert préentrainé

In [4]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
#This model only exists in PyTorch, so we use the _from_pt_ flag to import that model in TensorFlow
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)#, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
results=classifier("we love you")
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_57', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

label: LABEL_0, with score: 0.5104


## 3-sauvegarde du modèle en local sans le pusher dans le hub de huggingfaces

In [5]:
#save en local
model.save_pretrained('./test/', push_to_hub=False)
tokenizer.save_pretrained('./test/', push_to_hub=False)

('./test/tokenizer_config.json',
 './test/special_tokens_map.json',
 './test/vocab.txt',
 './test/added_tokens.json',
 './test/tokenizer.json')

## 4- relecture du modèle à partir du repertoire local -> ca fonctionne

In [6]:
#reload en local (attention )
model2=TFAutoModelForSequenceClassification.from_pretrained('./test')
tokenizer2 = AutoTokenizer.from_pretrained('./test')
classifier = pipeline("sentiment-analysis", model=model2, tokenizer=tokenizer2)
results=classifier("we love you")
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

Some layers from the model checkpoint at ./test were not used when initializing TFDistilBertForSequenceClassification: ['dropout_57']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./test and are newly initialized: ['dropout_77']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


label: LABEL_0, with score: 0.5104


## 5- connexion au workspace azure

In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.38.0 to work with nab-ML


## 6-azure model registration 

In [8]:
from azureml.core import Model
classification_model = Model.register(workspace=ws,
                       model_name='distilbert-sentiment',
                       model_path='./test',# local path
                       description='A text classification model')

Registering model distilbert-sentiment


## 7-lecture des modèles registrés

In [9]:
#lecture des modèles
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

distilbert-sentiment version: 1


sentiment_model_weight version: 1


sentiment_model version: 1




## 8-Creation du  folder pour deploiement

In [3]:
import os

# Create a folder for the deployment files
deployment_folder = './sentiment_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_sentiment.py'
script_path = os.path.join(deployment_folder,script_file)

./sentiment_service folder created.


## 9-ecriture du scoring script

In [16]:
%%writefile $script_path
import os
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
import json
from azureml.core import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model weight and load it
    #model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'distilbert-sentiment')
    model_path = Model.get_model_path('distilbert-sentiment')
    
    model2=TFAutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer2 = AutoTokenizer.from_pretrained(model_path)
    model = pipeline("sentiment-analysis", model=model2, tokenizer=tokenizer2)
    

# Called when a request is received
def run(text):
    return model(text)

Overwriting ./sentiment_service\score_sentiment.py


## 10- definition de l'environnement
* on recupere un environnement d'entrainement avec les bonnes dependances
* on ajoute la ligne de paramétrage d'inference

In [8]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model
from azureml.core import Workspace

ws = Workspace.from_config()
# Configure the scoring environment
#service_env = Environment.get(workspace=ws, name="customize_curated")
service_env = Environment.get(workspace=ws, name="customize_curated")
service_env.inferencing_stack_version = "latest"

In [ ]:
service_env

In [ ]:
from azureml.core.environment import CondaDependencies
#service_env = Environment(name='service-env')
#python_packages = ['transformers','os']
#,'pandas','string','os','azureml-defaults', 'azure-ml-api-sdk']
#conda_dep = CondaDependencies()
#for package in python_packages:
 #   service_env.python.conda_dependencies.add_pip_package(package)
#service_env.python.conda_dependencies=conda_dep

## 11- inference config et deploiement ok pour le distilbert préentrainé non finetuné

In [17]:
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)
model = ws.models['distilbert-sentiment']
# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4)

# Deploy the model as a service
print('Deploying model...')
service_name = "distilbert-sentiment"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-26 13:29:27+01:00 Creating Container Registry if not exists.
2022-02-26 13:29:27+01:00 Registering the environment.
2022-02-26 13:29:29+01:00 Use the existing image.
2022-02-26 13:29:29+01:00 Generating deployment configuration.
2022-02-26 13:29:31+01:00 Submitting deployment to compute.
2022-02-26 13:29:35+01:00 Checking the status of deployment distilbert-sentiment..
2022-02-26 13:37:44+01:00 Checking the status of inference endpoint distilbert-sentiment.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## 12- test du deploiement

In [18]:

# Call the web service
service.run("we love you")


[{'label': 'LABEL_0', 'score': 0.510405957698822}]

In [11]:
print(service.get_logs())

None


## 13- rechargement du modele distilbert pre entraine avec les poids finetunés

In [21]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

trained_model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)
    #compilation correpondant à l'entrainement
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
    #chargement des poids del'entrainement
trained_model.load_weights('./bert/mod_distil_sent_sw_full')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

## 14- sauvegarde en local du modele finetuné pour deploiement
* il suffit de remplacer le modele path du model registration

In [25]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.save_pretrained('./test2/', push_to_hub=False)
trained_model.save_pretrained('./test2/', push_to_hub=False)

In [26]:
mod = pipeline("sentiment-analysis", model=trained_model, tokenizer=tokenizer)

In [27]:
mod("we love you")

[{'label': 'LABEL_1', 'score': 0.8176223039627075}]

## 15- registration du modèle fine tuné

In [28]:
from azureml.core import Model
classification_model = Model.register(workspace=ws,
                       model_name='distilbert-sentiment-ft',
                       model_path='./test2',# local path
                       description='A text classification model')

Registering model distilbert-sentiment-ft


## 15- scoring script

In [29]:
import os

# Create a folder for the deployment files
deployment_folder = './sentiment_service_ft'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_sentiment.py'
script_path = os.path.join(deployment_folder,script_file)

./sentiment_service_ft folder created.


In [30]:
%%writefile $script_path
import os
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
import json
from azureml.core import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model weight and load it
    #model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'distilbert-sentiment')
    model_path = Model.get_model_path('distilbert-sentiment-ft')
    
    model2=TFAutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer2 = AutoTokenizer.from_pretrained(model_path)
    model = pipeline("sentiment-analysis", model=model2, tokenizer=tokenizer2)
    

# Called when a request is received
def run(text):
    return model(text)

Writing ./sentiment_service_ft\score_sentiment.py


## 16- deployment

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model
from azureml.core import Workspace

ws = Workspace.from_config()
# Configure the scoring environment
#service_env = Environment.get(workspace=ws, name="customize_curated")
service_env = Environment.get(workspace=ws, name="customize_curated")
service_env.inferencing_stack_version = "latest"

In [31]:
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)
model = ws.models['distilbert-sentiment-ft']
# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4)

# Deploy the model as a service
print('Deploying model...')
service_name = "distilbert-sentiment-ft"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-26 15:44:22+01:00 Creating Container Registry if not exists.
2022-02-26 15:44:22+01:00 Registering the environment.
2022-02-26 15:44:24+01:00 Use the existing image.
2022-02-26 15:44:25+01:00 Generating deployment configuration.
2022-02-26 15:44:26+01:00 Submitting deployment to compute.
2022-02-26 15:44:31+01:00 Checking the status of deployment distilbert-sentiment-ft..
2022-02-26 15:52:16+01:00 Checking the status of inference endpoint distilbert-sentiment-ft.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## 17- test du deploiement

In [32]:
# Call the web service
service.run("we love you")

[{'label': 'LABEL_1', 'score': 0.8176222443580627}]